In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate, Lambda,RepeatVector
from keras.optimizers import Adam

# Load your wind speed data
# Replace 'your_data.csv' with the actual path to your dataset
df = pd.read_csv('apple_stocks_with_sentiment10.csv', parse_dates=['Date'])
data = df[['Close']]

In [75]:
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self, # the constructor takes all the hyperparameters for the layer
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs): # the **kwargs argument takes care of all of the arguments for the parent class (input_shape, trainable, name)
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    # Block contains stack of 4 fully connected layers each has ReLU activation
    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    # Output of block is a theta layer with linear activation
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs): # the call method is what runs when the layer is called 
    x = inputs 
    for layer in self.hidden: # pass inputs through each hidden layer 
      x = layer(x)
    theta = self.theta_layer(x) 
    # Output the backcast and forecast from theta
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [76]:
Stock_prices=data.copy()

HORIZON = 1 # how far to predict forward
WINDOW_SIZE = 20 # how far to lookback

# Add windowed columns
Stock_prices_nbeats = Stock_prices.copy()
for i in range(WINDOW_SIZE):
  Stock_prices_nbeats[f"Price+{i+1}"] = Stock_prices_nbeats["Close"].shift(periods=i+1)
Stock_prices_nbeats.dropna().head()

# Make features and labels
X = Stock_prices_nbeats.dropna().drop("Close", axis=1)
y = Stock_prices_nbeats.dropna()["Close"]

# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(1596, 1596, 400, 400)

In [77]:
# 1. Turn train and test arrays into tensor Datasets
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

# 2. Combine features & labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 20), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 20), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [85]:
# Values from N-BEATS paper Figure 1 and Table 18/Appendix D
N_EPOCHS = 1000 # called "Iterations" in Table 18
N_NEURONS = 50 # called "Width" in Table 18
N_LAYERS = 8
N_STACKS = 8

INPUT_SIZE = WINDOW_SIZE * HORIZON # called "Lookback" in Table 18
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(20, 21)

In [86]:
from tensorflow.keras import layers

# 1. Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

# 2. Create input to stacks
stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

# 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
backcast, forecast = nbeats_block_layer(stack_input)
# Add in subtraction residual link, thank you to: https://github.com/mrdbourke/tensorflow-deep-learning/discussions/174 
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

# 4. Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

  # 5. Use the NBeatsBlock to calculate the backcast as well as block forecast
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals) # pass it in residuals (the backcast)

  # 6. Create the double residual stacking
  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# 7. Put the stack model together
model_7 = tf.keras.Model(inputs=stack_input, 
                         outputs=forecast, 
                         name="model_7_N-BEATS")

# 8. Compile with MAE loss and Adam optimizer
model_7.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])

# 9. Fit the model with EarlyStopping and ReduceLROnPlateau callbacks
model_7.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=2, # prevent large amounts of training outputs
            # callbacks=[create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch consumes far too much time
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=10, verbose=2)])

Epoch 1/1000
2/2 - 25s - loss: 17.8425 - mae: 17.8425 - mse: 344.5322 - val_loss: 16.1153 - val_mae: 16.1153 - val_mse: 291.1774 - lr: 0.0010 - 25s/epoch - 13s/step
Epoch 2/1000
2/2 - 0s - loss: 4.1743 - mae: 4.1743 - mse: 23.3695 - val_loss: 4.1322 - val_mae: 4.1322 - val_mse: 31.7887 - lr: 0.0010 - 115ms/epoch - 57ms/step
Epoch 3/1000
2/2 - 0s - loss: 1.5494 - mae: 1.5494 - mse: 3.5988 - val_loss: 4.6160 - val_mae: 4.6160 - val_mse: 28.8071 - lr: 0.0010 - 108ms/epoch - 54ms/step
Epoch 4/1000
2/2 - 0s - loss: 1.1449 - mae: 1.1449 - mse: 2.0013 - val_loss: 3.7373 - val_mae: 3.7373 - val_mse: 26.6012 - lr: 0.0010 - 126ms/epoch - 63ms/step
Epoch 5/1000
2/2 - 0s - loss: 1.8426 - mae: 1.8426 - mse: 4.8505 - val_loss: 4.4921 - val_mae: 4.4921 - val_mse: 27.4086 - lr: 0.0010 - 89ms/epoch - 44ms/step
Epoch 6/1000
2/2 - 0s - loss: 1.9077 - mae: 1.9077 - mse: 6.0200 - val_loss: 4.1821 - val_mae: 4.1821 - val_mse: 24.0169 - lr: 0.0010 - 86ms/epoch - 43ms/step
Epoch 7/1000
2/2 - 0s - loss: 1.1483

2/2 - 0s - loss: 0.6313 - mae: 0.6313 - mse: 0.7241 - val_loss: 1.6642 - val_mae: 1.6642 - val_mse: 5.3437 - lr: 0.0010 - 93ms/epoch - 46ms/step
Epoch 53/1000
2/2 - 0s - loss: 0.5556 - mae: 0.5556 - mse: 0.5650 - val_loss: 1.7307 - val_mae: 1.7307 - val_mse: 5.5264 - lr: 1.0000e-04 - 110ms/epoch - 55ms/step
Epoch 54/1000
2/2 - 0s - loss: 0.5560 - mae: 0.5560 - mse: 0.5636 - val_loss: 1.7564 - val_mae: 1.7564 - val_mse: 5.6002 - lr: 1.0000e-04 - 103ms/epoch - 52ms/step
Epoch 55/1000
2/2 - 0s - loss: 0.5542 - mae: 0.5542 - mse: 0.5604 - val_loss: 1.7165 - val_mae: 1.7165 - val_mse: 5.4620 - lr: 1.0000e-04 - 92ms/epoch - 46ms/step
Epoch 56/1000
2/2 - 0s - loss: 0.5491 - mae: 0.5491 - mse: 0.5529 - val_loss: 1.6574 - val_mae: 1.6574 - val_mse: 5.2761 - lr: 1.0000e-04 - 85ms/epoch - 42ms/step
Epoch 57/1000
2/2 - 0s - loss: 0.5489 - mae: 0.5489 - mse: 0.5545 - val_loss: 1.6194 - val_mae: 1.6194 - val_mse: 5.1644 - lr: 1.0000e-04 - 94ms/epoch - 47ms/step
Epoch 58/1000
2/2 - 0s - loss: 0.5500 

In [87]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

In [88]:
# Make predictions with N-BEATS model
model_7_preds = make_preds(model_7, test_dataset)
model_7_preds[:10]

1/1 [==============================] - 1s 1s/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([48.129738, 48.26116 , 48.369762, 48.513676, 48.53721 , 48.569527,
       48.833057, 48.95338 , 48.902355, 48.861748], dtype=float32)>

In [89]:
# MASE implementation
def mean_absolute_scaled_error(y_true, y_pred):
    mae = tf.reduce_mean(tf.abs(y_true-y_pred))
    
    # Find MAE of naive forecast
    mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:]-y_true[:-1]))
    
    return mae / mae_naive_no_season

In [90]:
def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)
  
  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}

In [91]:
# Evaluate N-BEATS model predictions
model_7_results = evaluate_preds(y_true=y_test,
                                 y_pred=model_7_preds)
model_7_results

{'mae': 1.534563,
 'mse': 5.243324,
 'rmse': 2.2898304,
 'mape': 2.744855,
 'mase': 1.6249967}